# Bu yarışmada **Komisyon Raporu** dosyaları için 500 belgenin incelendikten sonra bizden beklenilen görevler:

- "kategori" alanının tespiti. Bu görevle ilgili olan [Mevcut Çalışmamız](https://github.com/iflGARAJI10100/taMAM/blob/main/aramaProjesiKategoriBulmaKHK%20(1).ipynb) için tıklayınız.

- "donem" ve "sira_no" alanlarının tespiti. Bu kodda ise bu alanların tespitiyle alakalı çalışmalarımız mevcuttur.

Veri işleme ve analizi için pandas kütüphanesi kullanılacaktır. Pandas kütüphanesi pd rumuzu ile kısaltılmıştır.

Tarih verilerinin işlenmesi için **datetime** kütüphanesi kullanılacaktır.

In [1]:
import pandas as pd
from datetime import datetime

Verilerimiz Google Drive (bulut) üzerinde olduğu için Google Colab ile Google Drive arasında bağlantı kurulacaktır.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 isimli klasörde verilerimizi sakladığımız için bu klasöre erişilecektir.

***NOT:*** Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!

In [ ]:
%cd drive/
%cd MyDrive/
%cd Colab Notebooks/
%cd TDDI2022/

# Klasörün içeriğinin görülmesi için
%ls

Veri İskeleti TDDI2022 klasöründe bulunan yarışma kapsamında yarışmacılara sağlanan **kanunum-nlp-doc-analysis-dataset.csv** dosyasının içindeki verileri pandas ile okuyarak oluşturulacaktır.

Veri iskeleti oluşturulduktan sonra veri iskeleti ile ilgili bilgi ekrana yazdırılacaktır. Tam özetin yazdırılıp yazdırılmayacağı durumunu *verbose=True* (yazdırılsın) parametresi ile kontrol edilecektir. 

In [ ]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

Veri İskeletinde **kategori** sütununun / alanının kaç gruptan ve bu grupların kaç satırdan / kayıttan oluştuğu ekrana yazdırılacaktır.

In [ ]:
print(df.groupby('donem').size())

Verilerde yapacağımız ön çalışmada bizden istenilen değerleri araştıracağımız **Komisyon Raporu** belgelerinin özelliklerini inceliyoruz.

In [ ]:
print(df.loc[df['kategori'] == 'Komisyon Raporu'])

Verilerde yaptığımız ek çalışmalarda ise veri kümesinin data_text sütunundaki / alanındaki içerikte dağınık halde olduğu görülmüştür. Ama bazı belgelerde aranan kısımlar bozulabilmektedir. Bunun için Veri Temizleme İşlemleri Yapılmaktadır.

In [148]:
def metinSayi(exmetin):
  exmetin = exmetin.replace('S. Sayısı :.','S. Sayısı :')
  exmetin = exmetin.replace('S. Sayısı:','S. Sayısı :')
  exmetin = exmetin.replace('s. sAYisı :','S. Sayısı :')
  exmetin = exmetin.replace('S. Sayisi:','S. Sayısı :')
  exmetin = exmetin.replace('S. SAYISI :','S. Sayısı :')
  return exmetin

Yukarıdaki fonksiyonla verileri temzileyip düzelltikten sonra örnek bir belgeyi inceliyoruz.

In [ ]:
satir = 1345
metin = df['data_text'][satir]
metin = metinSayi(metin)

print('\nKategori: ',df['kategori'][satir],'\n','Dönem: ',df['donem'][satir],'\n','Sıra No: ',df['sira_no'][satir],'\n',35*'-','\nİçerik:\n',metin,sep='')

Örnek belgede de gözüktüğü üzere **aradığımız değerler** dosyada mevcuttur.

**Komisyon Raporu** kategorili toplam 500 belge mevcuttur. Belgelerden istenilen verilerin alınmasıyla ilgili takımımız belgelerde bulunan ortak yazım kurallarını tespit ederek ve **Kural Bazlı Arama** yapmayı tercih etmiştir.

Kodun devamında **bu yöntem** gösterilecek, **artı ve eksileri** hakkında tartışılacak ve yöntemin **başarı oranları** verilecektir.

# **Yöntem | Kural Bazlı Sistem ile Verilerin Eldesi**

Bu Yöntemde ise Belgeler arası **ortak yazım şekillerine** dikkat edilerek veriler elde edilir.

In [ ]:
# Sadece Komisyon Raporu kategorisindeki belgelerde işlem yapıyoruz.
degerlerimiz = ['donem','sira_no']
belgelerdekiToplamDegerSayisi = 500
belgelerdekiHataliDegerSayisi = [0] * len(degerlerimiz)


for i in range(1302,1802):
  # VERİ TEMİZLEME İŞLEMLERİ
  metin = df['data_text'][i]
  #metin = metinDonem(metin)

  # Asıl değerlerin alınması ve formatlanması
  dfDonem = df['donem'][i].split('.')[0]
  dfSiraNo = df['sira_no'][i].split(" ek ")
  
  # 'donem' değerini bulan kod 

  donem = ''
  if metin.find('Dönem') != -1:
    donemIndex = metin.find('Dönem')
    donemIndex = donemIndex + len('Dönem')

  elif metin.find('YASAMA YILI') != -1:
    donemIndex = metin.find('YASAMA YILI')
    donemIndex = donemIndex + len('YASAMA YILI')

  else:
    donem = '0'

  rakambuldu = 0
  while metin[donemIndex].isdigit() or rakambuldu==0 and donem != '0':
    if metin[donemIndex] == ' ' or metin[donemIndex] == '\n':
      rakambuldu = 0
    elif metin[donemIndex].isdigit():
      rakambuldu = 1
      donem = donem + metin[donemIndex]
    elif metin[donemIndex] == 'α':
      donem = ''
      break
    donemIndex += 1
    
  donem = int(donem)
    
  if int(dfDonem) >= 21 and donem != int(dfDonem):
    belgelerdekiHataliDegerSayisi[0]+=1
    print(i," DonemHatası ",'|',donem,'|',dfDonem,'|',sep='')

# 'sira_no' değerini bulan kod
  metin = metinSayi(metin)

  if metin.find('S. Sayısı :') != -1:
    siraNoIndex = metin.find('S. Sayısı :')
    siraNoIndex += len('S. Sayısı :')

  siraNo1 = ''
  rakambuldu = 0
  eskiIndex = 0
  while metin[siraNoIndex].isdigit() or rakambuldu==0:
    if metin[siraNoIndex] == ' ' or metin[siraNoIndex] == '\n':
      rakambuldu = 0

    elif metin[siraNoIndex].isdigit():
      rakambuldu = 1
      eskiIndex = siraNoIndex
      siraNo1 = siraNo1 + metin[siraNoIndex]
    siraNoIndex += 1


  siraNo2 = ''
  rakambuldu = 0
  count = 0

  if metin.find("ek",eskiIndex) != -1 and metin.find("ek",eskiIndex)-eskiIndex < 15:
    siraNoIndex = metin.find("ek",eskiIndex)
    siraNoIndex += len("ek")
    count = 1

    while metin[siraNoIndex].isdigit() or rakambuldu==0:
      if metin[siraNoIndex] == ' ' or metin[siraNoIndex] == '\n':
        rakambuldu = 0

      elif metin[siraNoIndex].isdigit():
        rakambuldu = 1
        count = 0
        siraNo2 = siraNo2 + metin[siraNoIndex]
      
      else:
        break

      siraNoIndex += 1

    if count == 1:
      siraNo2 = '1'

  if siraNo2 == '':
    siraNoToplam = siraNo1
  else:
    siraNoToplam = siraNo1 +' ek '+siraNo2

  dfSiraNo = df['sira_no'][i]

  if siraNoToplam != dfSiraNo:
    belgelerdekiHataliDegerSayisi[1]+=1
    print(i," SıraNoHatası ",'|',siraNoToplam,'|',dfSiraNo,'|',sep='')

for i in range(2):
  print(degerlerimiz[i],"|",belgelerdekiHataliDegerSayisi[i],"/",belgelerdekiToplamDegerSayisi,"|",round((belgelerdekiToplamDegerSayisi-belgelerdekiHataliDegerSayisi[i])/belgelerdekiToplamDegerSayisi,3))


## ***ÇIKARIM:***

- **(ÖNEMLİ BULGU)** Bu yöntemle modülümüz istenilen performans düzeyine yakınlaşmıştır.
- **(ÖNEMLİ BULGU)** Bu yöntemle modülümüz 'donem' alanının 500 değerinde 74 hatayla ve 0.85 Doğrulukla **Kabul Edilebilir Hatayla** bulmuştur.
- **(ÖNEMLİ BULGU)** Bu yöntemle modülümüz 'sira_sayisi' alanının 500 değerinde 131 hatayla ve 131 Doğrulukla **Kabul Edilebilir Hatayla** bulmuştur.

## **YÖNTEMİN ARTI VE EKSİLERİ**
### Artıları:
- Kural Bazlı Arama'sı diğer yöntemlere hız bakımından üstün.
- Kural Bazlı Arama'sı Kabul Edilebilir Doğruluk Performansıyla çalışıyor.

### Eksileri:
- Kural Bazlı Arama'nın diğer belge tiplerinde nasıl çalışacağı bilinmiyor.
- Kural Bazlı Arama'nın yeni yazım kurallarına göre yazılmış metinlerde Doğruluk oranı 0.0'a kadar düşebilir, riskli.

### **Yöntemin Performans Tablosu**

|Kural Bazlı Arama Yöntemi|Toplam|Doğru|Yanlış|Doğruluk Oranı|
|---|---|---|---|---|
|donem|500|426|74|0.852|
|sira_sayisi|500|369|131|0.738|